In [3]:
import requests
import json
import pandas as pd
from urllib.parse import quote
from tqdm import tqdm_notebook as tqdm

In [4]:
consumer_key = 'k3bxSsbVg9UK4nlHw5awgu1YEydyVFe5ru72zYbJ'
consumer_secret = 'nnjCIL5Tjb5Gx7KuBScek7ShtdR4KDEOyg0i5joA'

In [5]:
MY_API_KEY = consumer_key #  Replace this with your real API key

In [143]:
def request(host, path, api_key, url_params=None,verbose=False):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    
    if verbose:
        print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [141]:
def search(api_key, term, attribute=None, location=None, lat = None, \
           long=None,radius = None,verbose = False):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """
    
    if lat ==None or long == None:
        
        url_params = {
            'term': term.replace(' ', '+'),
            'location': location.replace(' ', '+'),
            'attributes':attribute.replace(' ', '+'),
            'radius':radius,
            'limit': SEARCH_LIMIT
        }
    else:
        url_params = {
            'term': term.replace(' ', '+'),
            'latitude':lat,
            'longitude':long,
            'radius':radius,
            'attributes':attribute.replace(' ', '+'),
            'limit': SEARCH_LIMIT
        }
    if verbose:
        print(url_params)
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params,verbose = verbose)

In [127]:
def reviews(api_key, id_biz):
    """Query the Reviews API by a business ID.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """
    REVIEWS_PATH = f'v3/businesses/{id_biz}/reviews'
    url_params = {
            'limit': None
        }
    print(REVIEWS_PATH)
    return request(API_HOST, REVIEWS_PATH, api_key, url_params=url_params)

In [122]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
SEARCH_LIMIT = 50

In [123]:
res = search(api_key,term = 'food', attribute='hot_and_new',lat = lat, long = lng, radius = 1000)

{'term': 'food', 'latitude': 37.840598, 'longitude': -122.254436, 'radius': 1000, 'attributes': 'hot_and_new', 'limit': 50}
Querying https://api.yelp.com/v3/businesses/search ...


In [ ]:
id_biz = '5TuE4wRGpwjQM-WcjDklpA'
reviews(api_key,id_biz);

In [115]:
for biz in res['businesses']:
    print(biz['distance'])

1011.9167215061187
811.3754405090626
1292.5865445127204
1641.6172004957943
875.4180880290585
1780.1066316725305
1756.4210293919255
1958.0961009098012


In [144]:
# get number of new restaurants in 15 min walking radius
num_new_business = []
for i in tqdm(range(df_tracts_locs.shape[0])):
    lat,lng = df_tracts_locs.INTPTLAT10[i],df_tracts_locs.INTPTLON10[i]
    res = search(api_key,term = 'food', attribute='hot_and_new',lat = lat, long = lng, radius = 1000)
    num_new_business.append(len(res['businesses']))

In [146]:
df_tracts_locs['new_businesses'] = num_new_business

In [147]:
df_tracts_locs.to_csv('../../data/processed/OpenData_Alameda/alameda_tracts_with_new_restaurants.csv')